In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# <center>Hive to BigQuery Migration
<table align="left">
<td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/dataproc-templates/blob/main/notebooks/hive2bq/HiveToBigquery_notebook.ipynb">
        <img src="../images/colab-logo-32px.png" alt="Colab logo" />Run in Colab
    </a>
</td>
<td>
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fdataproc-templates%2Fmain%2Fnotebooks%2Fhive2bq%2FHiveToBigquery_notebook.ipynb">
        <img src="../images/colab-enterprise-logo-32px.png" alt="GCP Colab Enterprise logo" />Run in Colab Enterprise
    </a>
</td>
<td>
    <a href="https://github.com/GoogleCloudPlatform/dataproc-templates/blob/main/notebooks/hive2bq/HiveToBigquery_notebook.ipynb">
        <img src="../images/github-logo-32px.png" alt="GitHub logo" />View on GitHub
    </a>
</td>
<td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/dataproc-templates/main/notebooks/hive2bq/HiveToBigquery_notebook.ipynb">
        <img src="../images/vertexai.jpg" alt="Vertex AI logo" />Open in Vertex AI Workbench
    </a>
</td>
</table>

#### References

- [DataprocPySparkBatchOp reference](https://google-cloud-pipeline-components.readthedocs.io/en/google-cloud-pipeline-components-1.0.0/google_cloud_pipeline_components.experimental.dataproc.html)
- [Kubeflow SDK Overview](https://www.kubeflow.org/docs/components/pipelines/sdk/sdk-overview/)
- [Dataproc Serverless in Vertex AI Pipelines tutorial](https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/ml_ops/stage3/get_started_with_dataproc_serverless_pipeline_components.ipynb)
- [Build a Vertex AI Pipeline](https://cloud.google.com/vertex-ai/docs/pipelines/build-pipeline)

This notebook is built to run a Vertex AI User-Managed Notebook using the default Compute Engine Service Account.  
Check the Dataproc Serverless in Vertex AI Pipelines tutorial linked above to learn how to setup a different Service Account.  

#### Permissions

Make sure that the service account used to run the notebook has the following roles:

- roles/aiplatform.serviceAgent
- roles/aiplatform.customCodeServiceAgent
- roles/storage.objectCreator
- roles/storage.objectViewer
- roles/dataproc.editor
- roles/dataproc.worker

#### Step 1:
#### Install the required packages

In [ ]:
# Google Cloud notebooks requires dependencies to be installed with '--user'
! pip3 install pyspark
! pip3 install --upgrade google-cloud-pipeline-components kfp --user -q
! pip3 install pip install google-auth==2.13.0
! pip3 install --upgrade google-cloud-bigquery-migration

# Install latest JDK
! sudo apt-get update
! sudo apt-get install default-jdk -y

#### Once you've installed the additional packages, you may need to restart the notebook kernel so it can find the packages.

Uncomment & Run this cell if you have installed anything from above commands

In [ ]:
# import os

# if not os.getenv("IS_TESTING"):
#    import IPython
#    app = IPython.Application.instance()
#    app.kernel.do_shutdown(True)

#### Step 2:
#### Set Google Cloud properties

**Overview**  
This notebook shows how to build a Vertex AI Pipeline to run a Dataproc Template   
using the DataprocPySparkBatchOp component.

In [ ]:
IS_PARAMETERIZED = False

In [ ]:
# User Configuration
# User Inputs
if not IS_PARAMETERIZED:
    get_project_id = ! gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT = get_project_id[0]
    REGION = ""  # example "us-west1"
    GCS_STAGING_LOCATION = "gs://<bucket_name>" # example "gs://my_bucket_name"
    SUBNET = "" # example "projects/<project-id>/regions/<region-id>/subnetworks/<subnet-name>" 
    INPUT_HIVE_DATABASE= ""
    INPUT_HIVE_TABLES= "*" # example "table1,table2,table3..." or "*"
    OUTPUT_BIGQUERY_DATASET= ""
    TEMP_BUCKET= "<bucket_name>"
    HIVE_METASTORE= "" # example "thrift://<my_hive_server>:9083"
    BQ_DATASET_REGION="us"
    DATAPROC_SERVICE_ACCOUNT = "" # eg: test@project_id.iam.gserviceaccount.com

    ## Change if needed
    HIVE_OUTPUT_MODE="overwrite"
    MAX_PARALLELISM=10 # Controls number of parallel Dataproc Serverless Jobs

#### Step 3:
#### Import dependencies

In [ ]:
import google.cloud.aiplatform as aiplatform
from kfp import dsl
from kfp import compiler
from datetime import datetime

try:
    from google_cloud_pipeline_components.experimental.dataproc import DataprocPySparkBatchOp
except ModuleNotFoundError:
    from google_cloud_pipeline_components.v1.dataproc import DataprocPySparkBatchOp

import time
import os
import sys
from pyspark.sql import SparkSession
import pandas as pd
from pathlib import Path
import subprocess
from google.cloud import bigquery

module_path = os.path.abspath(os.pardir)
if module_path not in sys.path:
    sys.path.append(module_path)

from util.sql_translation import create_migration_workflow, get_migration_workflow_status

#### Step 4:
#### Change working directory to the Dataproc Templates python folder

In [ ]:
cur_path = Path(os.getcwd())
if IS_PARAMETERIZED:
    WORKING_DIRECTORY = os.path.join(cur_path.parent ,'python')
else:
    WORKING_DIRECTORY = os.path.join(cur_path.parent.parent ,'python')

# If the above code doesn't fetches the correct path please
# provide complete path to python folder in your dataproc 
# template repo which you cloned 

# WORKING_DIRECTORY = "/home/jupyter/dataproc-templates/python/"
print(WORKING_DIRECTORY)

In [ ]:
%cd $WORKING_DIRECTORY

#### Step 5:
#### Build Dataproc Templates python package

In [ ]:
PACKAGE_EGG_FILE = "dist/dataproc_templates_distribution.egg"
! python ./setup.py bdist_egg --output=$PACKAGE_EGG_FILE

#### Step 6:
#### Copy package to the Cloud Storage bucket

For this, make sure that the service account used to run the notebook has the following roles:
 - roles/storage.objectCreator
 - roles/storage.objectViewer

In [ ]:
! gcloud storage cp main.py $GCS_STAGING_LOCATION/
! gcloud storage cp $PACKAGE_EGG_FILE $GCS_STAGING_LOCATION/dist/

#### Step 7:
#### Get Hive Tables 
In case user wants to load all the Hive tables from the database, we need to get the table list using the metastore.

Below cell will fetch all tables from the Hive database by running a Spark SQL query using the provided Hive Metastore.

In [ ]:
if INPUT_HIVE_TABLES=="*":
    #os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
    #os.environ["PATH"] = os.environ["JAVA_HOME"] + "/bin:" + os.environ["PATH"]
    spark=SparkSession.builder \
          .master("local")\
          .appName("Spark Job to get HIVE table list") \
          .config("hive.metastore.uris",HIVE_METASTORE) \
          .enableHiveSupport() \
          .getOrCreate()  
    TABLE_LIST_DF=spark.sql("show tables in "+INPUT_HIVE_DATABASE)
    TABLE_LIST=TABLE_LIST_DF.select("tableName").rdd.flatMap(lambda x: x).collect()
    print("Table Sets to Migrate: ")
    print(TABLE_LIST)
    spark.stop()
else:
    TABLE_LIST=INPUT_HIVE_TABLES.split(",")
    print("Table Sets to Migrate: ")
    print(TABLE_LIST)

#### -----  Skip steps 8-16 to create all HIVE tables non partitioned tables in Bigquery -----


#### Step 8:
#### Get Required Variables for HIVEDDLEXTRACTOR

In [ ]:
DDL_INPUT_PATH=GCS_STAGING_LOCATION+"/hiveddl/input"
DDL_OUTPUT_PATH=GCS_STAGING_LOCATION+"/hiveddl/output"
os.environ["GCP_PROJECT"]=PROJECT
os.environ["REGION"]=REGION
os.environ["GCS_STAGING_LOCATION"]=GCS_STAGING_LOCATION
os.environ["SUBNET"]=SUBNET
os.environ["HIVE_METASTORE"]=HIVE_METASTORE
os.environ["INPUT_HIVE_DATABASE"]=INPUT_HIVE_DATABASE
os.environ["GCS_STAGING_PATH"]=DDL_INPUT_PATH

#### Step 9:
#### Run HIVEDDLEXTRACTOR to extract HIVE DDLs

We will be making use of HIVEDDLEXTRACTOR utility to connect to thrift server and extracting all the DDLs. 

In [ ]:
!./bin/start.sh \
    --properties=spark.hadoop.hive.metastore.uris=$HIVE_METASTORE \
    -- --template=HIVEDDLEXTRACTOR \
    --hive.ddl.extractor.input.database=$INPUT_HIVE_DATABASE \
    --hive.ddl.extractor.output.path=$GCS_STAGING_PATH


#### Step 10:
#### Get the latest DDL Cloud Storage path

HIVEDDLEXTRACTOR utility will create seperate directory for each run based on the hive database name and run time.

In [ ]:
result=subprocess.run(f"gcloud storage ls  {DDL_INPUT_PATH}/{INPUT_HIVE_DATABASE} | sed '$!d'", capture_output=True, shell=True, encoding="utf-8")
latest_ddl_path=str(result.stdout)[:-1]
print(latest_ddl_path)

#### Step 12:
#### Copy Global Typeconvert file into the input folder

Currently Translation API converts all the TIMESTAMP datatypes to DATETIME, Spark faces difficult copying TIMESTAMP Hive column to DATETIME Bigquery column. Global Typeconvert configuration will convert back all the DATETIME column to TIMESTAMP

In [ ]:
! gcloud storage cp ../notebooks/util/global_typeconvert.config.yaml "$latest_ddl_path"

#### Step 13:
#### Create object name mapping

Object name mapping will help us to replace HIVE database name with Bigquery Dataset in the final DDL

In [ ]:
obj_name_mapping={
                    "name_map": [{
                      "source": {
                        "schema": INPUT_HIVE_DATABASE,
                                 },
                      "target": {
                        "schema": OUTPUT_BIGQUERY_DATASET,
                                 }
                                }, 
                                ]
                  }

#### Step 14:
#### Call create_migration_workflow to convert HIVE DDls to Bigquery syntax

Below cell will call create_migration_workflow function to call BQ Translation API and then wait for the status to be completed.

It will create the translated DDLs in "gs://bucket_name/hiveddl/output/bq_dataset"

In [ ]:
#Set required variables to be passed to the create_migration_workflow
gcs_input_path=latest_ddl_path
gcs_output_path=DDL_OUTPUT_PATH+"/"+OUTPUT_BIGQUERY_DATASET
project_id=PROJECT
bq_dataset=OUTPUT_BIGQUERY_DATASET
default_database=PROJECT
source_dilect="hive"
bq_region=BQ_DATASET_REGION

# Call create_migration_workflow with the required parameters
workflow_name,workflow_state=create_migration_workflow(
    gcs_input_path, gcs_output_path, project_id, bq_dataset,
    default_database, source_dilect, bq_region,obj_name_mapping)

#Get Workflow status
while (str(workflow_state) == "State.RUNNING"):
    print("Running Migration Workflow")
    time.sleep(5)
    workflow_state=get_migration_workflow_status(workflow_name).state
print(str(workflow_state))

#### Step 14:
#### Extract translated DDLs

Below cell will read all the translated Bigquery DDLs

In [ ]:
from google.cloud import storage
result=subprocess.run(f"gcloud storage ls  {gcs_output_path} ", capture_output=True, shell=True, encoding="utf-8")
translated_files=result.stdout.split("\n")
all_ddls=""
client = storage.Client()
for file in translated_files:
    if "/_SUCCESS" not in file and "batch_translation_report.csv" not in file and "consumed_name_map.json" not in file and len(file)>0:
        bucket_name=file.replace("gs://","").split("/",1)[0]
        file_path=file.replace("gs://","").split("/",1)[1]
        bucket = client.get_bucket(bucket_name)
        blob = bucket.blob(file_path)
        all_ddls=all_ddls+(blob.download_as_string().decode("utf-8"))

#### Step 15:
#### Run translated ddls and create BQ partitioned and clustered tables

Below cell will try to run each bigquery DDL one by one and save the status in DDL_FAIL_AUDIT_DF dataframe

In [ ]:
DDL_FAIL_AUDIT_DICT={}
DDL_FAIL_AUDIT_DF = pd.DataFrame(columns=["Source_DB_Name","Source_Table_Set","Target_DB_Name","Target_Table_Set","Job_Start_Time","Job_End_Time","Job_Status"])
client = bigquery.Client()
for ddl in all_ddls[:-1].split(";\n"):
    tblnm=ddl.split("CREATE TABLE")[1].split("\n")[0].split(".")[2]
    try:
        job = client.query(ddl)
        job.result()
        print(f"Table Created in bigquery: {OUTPUT_BIGQUERY_DATASET}.{tblnm}")
    except Exception as e:
        print(f"Failed to create table: {OUTPUT_BIGQUERY_DATASET}.{tblnm}")
        print(e)
        DDL_FAIL_AUDIT_DICT["Source_DB_Name"]=INPUT_HIVE_DATABASE
        DDL_FAIL_AUDIT_DICT["Source_Table_Set"]=tblnm
        DDL_FAIL_AUDIT_DICT["Target_DB_Name"]=OUTPUT_BIGQUERY_DATASET
        DDL_FAIL_AUDIT_DICT["Target_Table_Set"]=tblnm
        DDL_FAIL_AUDIT_DICT["Job_Start_Time"]=str(datetime.now())
        DDL_FAIL_AUDIT_DICT["Job_Status"]=f"FAIL REASON: {e}"
        DDL_FAIL_AUDIT_DICT["Job_End_Time"]=str(datetime.now())
        DDL_FAIL_AUDIT_DF=DDL_FAIL_AUDIT_DF.append(DDL_FAIL_AUDIT_DICT, ignore_index = True)


#### Step 16:
#### Remove failed tables from the final TABLE_LIST


In [ ]:
import copy
FAILED_DDL_TBLS=DDL_FAIL_AUDIT_DF[['Target_Table_Set']].values.ravel().tolist()
TABLE_LIST_COPY=copy.deepcopy(TABLE_LIST)
for element in TABLE_LIST_COPY:
    if element in FAILED_DDL_TBLS:
        TABLE_LIST.remove(element)



#### Step 17:

Split Hive Tables list based on MAX_PARALLELISM value provided by the user.

In [ ]:
COMPLETE_LIST = copy.deepcopy(TABLE_LIST)
PARALLEL_JOBS = len(TABLE_LIST)//MAX_PARALLELISM
JOB_LIST = []
while len(COMPLETE_LIST) > 0:
    SUB_LIST = []
    for i in range(MAX_PARALLELISM):
        if len(COMPLETE_LIST)>0 :
            SUB_LIST.append(COMPLETE_LIST[0])
            COMPLETE_LIST.pop(0)
        else:
            break
    JOB_LIST.append(SUB_LIST)
print("List of tables for execution : ")
print(JOB_LIST)

#### Step 18:

Set Dataproc Template Properties

In [ ]:
PIPELINE_ROOT = GCS_STAGING_LOCATION + "/pipeline_root/dataproc_pyspark"
MAIN_PYTHON_FILE = GCS_STAGING_LOCATION + "/main.py"
JARS = ["gs://spark-lib/bigquery/spark-bigquery-latest_2.12.jar"]
PYTHON_FILE_URIS = [GCS_STAGING_LOCATION + "/dist/dataproc_templates_distribution.egg"]

#### Step 19:
#### Build pipeline and run Dataproc Template on Vertex AI Pipelines to migrate Hive tables to BigQuery

For this, make sure that the service account used to run the notebook has the following roles:
 - roles/dataproc.editor
 - roles/dataproc.worker

In [ ]:
runtime_prop={}
runtime_prop['spark.hadoop.hive.metastore.uris']=HIVE_METASTORE
runtime_prop['mapreduce.fileoutputcommitter.marksuccessfuljobs'] = "false"

def migrate_hive(EXECUTION_LIST):
    EXECUTION_LIST = EXECUTION_LIST
    aiplatform.init(project=PROJECT, staging_bucket=GCS_STAGING_LOCATION)

    @dsl.pipeline(
        name="hive-to-bq-pyspark",
        description="Pipeline to migrate tables from hive to bq",
    )
    def pipeline(
        project_id: str = PROJECT,
        location: str = REGION,
        main_python_file_uri: str = MAIN_PYTHON_FILE,
        python_file_uris: list = PYTHON_FILE_URIS,
        jar_file_uris: list = JARS,
        subnetwork_uri: str = SUBNET,
        service_account: str = DATAPROC_SERVICE_ACCOUNT
    ):
        for table in EXECUTION_LIST:
            BATCH_ID = "hive2bq-{}-{}".format(table,datetime.now().strftime("%s")).replace('_','-')
            TEMPLATE_SPARK_ARGS = [
                                    "--template=HIVETOBIGQUERY",
                                    "--hive.bigquery.input.database={}".format(INPUT_HIVE_DATABASE),
                                    "--hive.bigquery.input.table={}".format(table),
                                    "--hive.bigquery.output.table={}".format(table),
                                    "--hive.bigquery.output.dataset={}".format(OUTPUT_BIGQUERY_DATASET),
                                    "--hive.bigquery.output.mode={}".format(HIVE_OUTPUT_MODE),
                                    "--hive.bigquery.temp.bucket.name={}".format(TEMP_BUCKET)                                    ]
            _ = DataprocPySparkBatchOp(
                project=project_id,
                location=location,
                batch_id=BATCH_ID,
                main_python_file_uri=main_python_file_uri,
                python_file_uris=python_file_uris,
                jar_file_uris=jar_file_uris,
                subnetwork_uri=subnetwork_uri,
                service_account=service_account,
                runtime_config_properties=runtime_prop,
                runtime_config_version="1.1", # issue 665
                args=TEMPLATE_SPARK_ARGS,
            )
            time.sleep(5)

    compiler.Compiler().compile(pipeline_func=pipeline, package_path="pipeline.json")

    pipeline = aiplatform.PipelineJob(
            display_name="pipeline",
            template_path="pipeline.json",
            pipeline_root=PIPELINE_ROOT,
            enable_caching=False,
            location=REGION,
            )
    # run() method has an optional parameter `service_account` which you can pass if you want to run pipeline using
    # specific service account instead of default service account 
    # eg. pipeline.run(service_account='test@project_id.iam.gserviceaccount.com')
    pipeline.run()

#### Step 20:

Run Dataproc Batch Template based on Hive Tables list calculated in Step 8.

The below cell will call function migrate_hive to migrate tables using dataproc serverless batch job and also add an entry in Audit Table for each Table Set.

In [ ]:
AUDIT_DICT={}
AUDIT_DF = pd.DataFrame(columns=["Source_DB_Name","Source_Table_Set","Target_DB_Name","Target_Table_Set","Job_Start_Time","Job_End_Time","Job_Status"])
 
for execution_list in JOB_LIST:
    print("\n\nLoading Table Set: "+str(execution_list))
    AUDIT_DICT["Source_DB_Name"]=INPUT_HIVE_DATABASE
    AUDIT_DICT["Source_Table_Set"]='|'.join(execution_list)
    AUDIT_DICT["Target_DB_Name"]=OUTPUT_BIGQUERY_DATASET
    AUDIT_DICT["Target_Table_Set"]='|'.join(execution_list)
    AUDIT_DICT["Job_Start_Time"]=str(datetime.now())
    try:
        migrate_hive(execution_list)
    
    except Exception as e:
        AUDIT_DICT["Job_Status"]="FAIL"
        raise Exception("\n\nSome Error Occured while loading Table Set: "+str(execution_list)) from e
    else:
        AUDIT_DICT["Job_Status"]="PASS"
        print("\n\nLoaded Table Set: "+str(execution_list))

    AUDIT_DICT["Job_End_Time"]=str(datetime.now())
    AUDIT_DF=AUDIT_DF.append(AUDIT_DICT, ignore_index = True)

AUDIT_DF_COMBINED = pd.concat([AUDIT_DF, DDL_FAIL_AUDIT_DF], axis=0)

if AUDIT_DF_COMBINED.empty:
    print("Audit Dataframe is Empty")
else:
    print(AUDIT_DF_COMBINED)
    AUDIT_DF_COMBINED.to_csv("gs://"+TEMP_BUCKET+"/audit/audit_file_{}.csv".format(str(datetime.now())),index=False,header = False)